In [ ]:
# Random Imports
import os
import sys
import pathlib
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'train.ipynb'

from datetime import datetime
# Get today's date
today_date = datetime.now()
# Format the date as MM_DD_YY
formatted_date = today_date.strftime("%m_%d_%y")

# Setup direcrtories
root = pathlib.Path('/storage/vbutoi/scratch/ESE')

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

## Define said jobs

In [ ]:
%%yaml default_cfg 

experiment:
  seed: '?' 
    
dataloader:
  batch_size: '?' 
  num_workers: '?' 
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 3.0e-4
  weight_decay: 0.0 
  
train:
  epochs: 1000 
  eval_freq: 5 
  # pretrained_dir: None # In case we want to load a pretrained model.

log:
  checkpoint_freq: 20 
  root: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      from_logits: True
      batch_reduction: 'mean' 
      ignore_empty_labels: False 
      ignore_index: 0 # Ignore background class when reporting.

######################
# Cross-Entropy Loss #
######################
loss_func: 
  _class: ionpy.loss.PixelCELoss
  from_logits: True
  batch_reduction: 'mean' 
  
#############
# Dice Loss #
#############
# loss_func: 
#   _class: ionpy.loss.SoftDiceLoss
#   from_logits: True
#   batch_reduction: 'mean' 
#   ignore_empty_labels: False 
#   ignore_index: 0 

In [ ]:
%%yaml model_cfg  

model:
  _class: ese.experiment.models.unet.UNet
  filters: [64, 64, 64, 64, 64]
  convs_per_block: 3

In [ ]:
%%yaml dataset_cfg 

########
## WMH #
########
# data:
#   _class: ese.experiment.datasets.WMH
#   axis: 0
#   task: Amsterdam
#   slicing: dense 
#   annotator: observer_o12
#   num_slices: 1
#   in_channels: 1
#   out_channels: 2 
#   version: 0.2
#   iters_per_epoch: 1000 
#   preload: '?' 

###################
## OASIS 4-Labels #
###################
# data:
#   _class: ese.experiment.datasets.OASIS
#   axis: 0
#   label_set: label4
#   slicing: central 
#   num_slices: 1
#   in_channels: 1
#   out_channels: 5 
#   central_width: 32
#   version: 0.1
#   preload: False 

###################
## OASIS Thalamus #
###################
# data:
#   _class: ese.experiment.datasets.OASIS
#   axis: 1
#   label_set: label35
#   num_slices: 1
#   in_channels: 1
#   central_width: 32
#   version: 0.2
#   out_channels: 2 
#   binary: True 
#   target_labels: '?'
#   iters_per_epoch: 1000 
#   preload: '?' 

# Labels for the thalamus
# - 7
# - 26 

# Labels for the hippocampus
# - 14
# - 30

###############
## CityScapes #
###############
data:
  _class: ese.experiment.datasets.CityScapes
  in_channels: 3
  out_channels: 19 

###############
## OxfordPets #
###############
# data:
#   _class: ese.experiment.datasets.OxfordPets
#   preload: True
#   version: 0.1
#   num_classes: all 
#   in_channels: 3
#   out_channels: 38 # 37 + 1 (background)

######################
## Binary OxfordPets #
######################
# data:
#   _class: ese.experiment.datasets.BinaryPets
#   preload: True
#   version: 0.1
#   in_channels: 3
#   out_channels: 2

In [ ]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

In [ ]:
%%yaml aug_cfg 

# AUGMNENTATIONS USED FOR CITYSCAPES
augmentations:
    train:
        - RandomCrop:
            height: 256
            width: 512
        - HorizontalFlip:
            p: 0.5
    val:
        - Resize:
            height: 256
            width: 512

## Debug Station

In [ ]:
# Setup the root.
exp_name = '01_24_24_CityScapes_CrossEntropy_Ensemble'
# add_string = '_HippocampusTraining'
# exp_root = str(root / 'training' / (formatted_date + add_string) / exp_name)
exp_root = str(root / 'training' / exp_name)

# Create the ablation options
option_set = [
    {
        'log.root': [exp_root],
        'data.preload': [False],
        'dataloader.batch_size': [8],
        'dataloader.num_workers': [2],
        'experiment.seed': [40, 41, 42, 43]
    }
]

In [ ]:
import copy
from ionpy.util import Config
from ionpy.util.config import check_missing
from ese.scripts.utils import get_option_product

# Assemble base config
base_cfg = Config(default_cfg).update([model_cfg, dataset_cfg, callbacks_cfg])

# Get the configs
raw_cfgs = get_option_product(exp_name, option_set, base_cfg)

cfgs = []
for cfg in raw_cfgs:
    cfg = cfg.update(aug_cfg)
    check_missing(cfg) # Verify there are no ? in config.
    cfgs.append(cfg)

In [ ]:
len(cfgs)

## Running Jobs

In [ ]:
# ######## FOR DEBUGGING
# from ese.experiment.experiment import run_ese_exp, CalibrationExperiment

# run_ese_exp(
#     config=cfgs[0], 
#     experiment_class=CalibrationExperiment,
#     gpu='1',
#     run_name='debug',
#     show_examples=False,
#     track_wandb=False
# )

In [ ]:
# FOR SUBMISSION
from ese.experiment.experiment import submit_ese_exps, CalibrationExperiment 

submit_ese_exps(
    exp_root=exp_root,
    experiment_class=CalibrationExperiment,
    config_list=cfgs,
    available_gpus=['0', '1', '2', '3'],
    # available_gpus=['3', '4', '5', '6', '7'],
    track_wandb=True
)